# Phase 1 — Python EDA Notebook

**Dataset:** `e-commerce dataset1.csv`

This notebook performs a full exploratory data analysis (EDA) for Phase 1. Run cells sequentially.

In [ ]:
# Imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (8,4)
OUTPUT_FIG_DIR = "../figures"
os.makedirs(OUTPUT_FIG_DIR, exist_ok=True)


## 1. Load dataset
Make sure this notebook resides in `Ecommerce_project/02_python/` so the relative path to the data folder works.

In [ ]:
# Load dataset
DATA_PATH = "../Data/e-commerce dataset1.csv"
df = pd.read_csv(DATA_PATH, low_memory=False)
print("Loaded:", DATA_PATH)
print("Shape:", df.shape)
df.head()

## 2. Columns and data types

In [ ]:
for c in df.columns:
    print(repr(c))
print("\nData types:")
display(df.dtypes)

## 3. Missing values and duplicates

In [ ]:
missing = df.isna().sum().to_frame('missing_count')
missing['missing_pct'] = (missing['missing_count'] / len(df) * 100).round(3)
display(missing.sort_values('missing_pct', ascending=False))
print("Duplicate rows:", df.duplicated().sum())

## 4. Clean / normalize column names (safe to run multiple times)

In [ ]:
import re
def clean_col(c):
    c = c.strip()
    c = c.replace('%','pct')
    c = re.sub(r'[^\w\s]', '', c)
    c = re.sub(r'\s+', '_', c)
    return c.lower()
df.columns = [clean_col(c) for c in df.columns]
print("New columns:")
print(df.columns.tolist())

## 5. Convert date column (`purchase_date`) to datetime (handles mixed formats)

In [ ]:
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='mixed', dayfirst=True, errors='coerce')
print("Converted purchase_date, non-null count:", df['purchase_date'].notna().sum())
bad_dates = df[df['purchase_date'].isna()]
if not bad_dates.empty:
    print("Rows with unparsed dates (showing up to 10):")
    display(bad_dates.head(10))


## 6. Detect numeric-like columns and convert to numeric

In [ ]:
candidates = [c for c in df.columns if any(k in c for k in ['price','final','discount','amount','cost','total'])]
print("Numeric candidates:", candidates)
def clean_numeric_series(s):
    return pd.to_numeric(
        s.astype(str)
         .str.replace(',', '', regex=False)
         .str.replace('₹', '', regex=False)
         .str.replace('rs', '', case=False, regex=False)
         .str.replace('%', '', regex=False)
         .str.strip(),
        errors='coerce'
    )
for col in candidates:
    df[col] = clean_numeric_series(df[col])
if candidates:
    display(df[candidates].describe().T)
else:
    print("No numeric-like columns auto-detected.")

## 7. Basic statistical summary and unique counts

In [ ]:
display(df.describe(include='all').T)
print("\nUnique counts:")
display(df.nunique().to_frame('unique_count'))

## 8. Outlier detection using IQR (shows counts and examples)

In [ ]:
outlier_info = {}
for c in candidates:
    ser = df[c].dropna()
    if ser.empty:
        outlier_info[c] = {'count': 0}
        continue
    q1 = ser.quantile(0.25)
    q3 = ser.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    mask = (df[c] < lower) | (df[c] > upper)
    outlier_info[c] = {'count': int(mask.sum()), 'lower': float(lower), 'upper': float(upper)}
    print(f"{c}: {outlier_info[c]['count']} outliers (bounds: {lower:.4f} - {upper:.4f})")
    display(df.loc[mask, [c]].head(10))
outlier_info

## 9. Time-based features & Monthly sales trend

In [ ]:
df['month'] = df['purchase_date'].dt.to_period('M')
df['year'] = df['purchase_date'].dt.year
final_col = next((c for c in candidates if 'final' in c), None)
price_col = next((c for c in candidates if 'price' in c and 'final' not in c), None)
use_col = final_col or price_col
print("Using column for sales aggregation:", use_col)
monthly = df.groupby('month')[use_col].sum().reset_index().sort_values('month')
display(monthly.head(), monthly.tail())
if not monthly.empty:
    plt.figure(figsize=(10,4))
    plt.plot(monthly['month'].astype(str), monthly[use_col], marker='o')
    plt.xticks(rotation=45)
    plt.title('Monthly Sales Trend')
    plt.ylabel('Total Sales (Rs.)')
    plt.xlabel('Month')
    plt.tight_layout()
    plt.savefig(OUTPUT_FIG_DIR + '/monthly_sales.png')
    plt.show()
else:
    print("No monthly data to plot.")

## 10. Top products by count and by revenue

In [ ]:
prod_col = 'product_id' if 'product_id' in df.columns else None
if prod_col:
    top_count = df[prod_col].value_counts().head(10).rename_axis('product_id').reset_index(name='count')
    display(top_count)
else:
    print("No product_id column detected.")
if use_col and prod_col:
    top_revenue = df.groupby(prod_col)[use_col].sum().sort_values(ascending=False).head(10).reset_index()
    display(top_revenue)
else:
    print("Cannot compute revenue per product (missing columns).")

## 11. Purchase frequency & customer summary

In [ ]:
user_col = 'user_id' if 'user_id' in df.columns else None
if user_col and use_col:
    cust = df.groupby(user_col).agg(
        num_orders = (prod_col,'count') if prod_col else (user_col,'count'),
        total_value = (use_col,'sum'),
        first_purchase = ('purchase_date','min'),
        last_purchase = ('purchase_date','max')
    ).reset_index().sort_values('num_orders', ascending=False)
    display(cust.head(10))
else:
    print("Cannot compute customer summary (missing columns).")

## 12. Payment method analysis

In [ ]:
pm_col = 'payment_method' if 'payment_method' in df.columns else None
if pm_col:
    pm = df[pm_col].value_counts().to_frame('count')
    pm['pct'] = (pm['count'] / pm['count'].sum() * 100).round(2)
    display(pm)
    pm.plot.pie(y='count', autopct='%1.1f%%', legend=False, ylabel='')
    plt.title('Payment Method Share')
    plt.tight_layout()
    plt.savefig(OUTPUT_FIG_DIR + '/payment_method_share.png')
    plt.show()
else:
    print("No payment_method column detected.")

## 13. Correlation matrix (numeric columns)

In [ ]:
numdf = df[candidates].copy() if candidates else pd.DataFrame()
if not numdf.empty and numdf.shape[1] > 1:
    corr = numdf.corr()
    display(corr)
    plt.figure(figsize=(6,5))
    plt.matshow(corr, fignum=1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.colorbar()
    plt.title('Correlation matrix', pad=40)
    plt.tight_layout()
    plt.savefig(OUTPUT_FIG_DIR + '/correlation_matrix.png')
    plt.show()
else:
    print("Not enough numeric columns for correlation.")

## 14. Save cleaned dataset and final notes

In [ ]:
OUT_CLEAN = "../Data/cleaned_e-commerce_dataset1.csv"
df.to_csv(OUT_CLEAN, index=False)
print("Saved cleaned dataset to:", OUT_CLEAN)
print("\nEDA completed. Check generated figures in the figures/ folder and cleaned CSV in the Data/ folder.")